# P-SAT: DETECTING COMPANY REFERENCES IN THE PRESS

In [22]:
# IMPORTS
from tools.utility import *
from articleClassifier import ArticleClassifier 

import json
from sklearn.model_selection import train_test_split

## Fetch Tokenized Cleaned Corpus of Articles

### Load Tokenized Corpus

In [33]:
PATH = "./data/corpus/"
file = "cleaned_tokenized_corpus"
with open(PATH + file +".json") as json_file: 
    article_corpus = json.load(json_file) 

### Count number of articles per company

In [15]:
dict_count = dict()
for document in article_corpus:
    labels = document["siren"]
    for siren in labels:
        if siren in dict_count.keys():
            dict_count[siren] +=1
        else:
            dict_count[siren] = 1

### List companies with more than N articles

In [18]:
n_associated_articles = 5
siren_filtered =[key for key in dict_count if dict_count[key] > n_associated_articles]
print ("There are",len(siren_filtered),"companies with MORE than",n_associated_articles,"associated articles")

There are 1501 companies with MORE than 5 associated articles


## Split Corpus into Train and Test sets

In [31]:
test_size = 0.3
X_train_corpus, X_test_corpus = train_test_split(corpus, test_size=test_size, random_state=0)
X_test_corpus = clean_corpus_labels (X_test_corpus, siren_filtered) # Removing labels we are not considering

Number of labels removed: 0


## Extract Scored Relevant Words from dictionnairy

### OPTION 1: generate the relevant words

In [ ]:
corpus = X_train_corpus     # corpus
list_siren = siren_filtered # companies kept for the study
relevant_words_tfidf = generate_relevant_words_tfidf(corpus,list_siren)
#100%|██████████| 1501/1501 [1:21:23<00:00,  2.34s/it] # nouns sublinear_tf

### OPTION 2: load pre-generated words

In [24]:
# load dictionary 
PATH = "./data/relevant_words/"
file = "rw_tfidf_sublinear_tf_1501_companies"
a_file = open(PATH + file + ".json", "r")
relevant_words_tfidf = json.load(a_file)
print (file,"is loaded successfully") 

rw_tfidf_sublinear_tf_1501_companies is loaded successfully


## Predict Companies of the Test Set

In [ ]:
# HYPERPARAMETERS
related_words = relevant_words_tfidf #model related words
corpus = X_test_corpus[:1000]
#corpus = X_test_corpus
min_score = 0.9
n_sig_words = 3
t = 250
max_n_pred = None
criterion = "T"

# FIT and PREDICT of ArticleClassifier model
ac_model = ArticleClassifier(n_sig_words,min_score ,t) # Init Article Classifier 
ac_model.fit(related_words)    # fit related words
predictions = ac_model.predict(corpus,max_n_pred, criterion) # evaluate corpus
print ("Predictions on Test Set Complete")

## Evaluate Prediction

In [ ]:
ac_model.evaluate()
ac_model.print_eval(verbose = 2) # verbose = 0,1,2

# Demonstrations on ArticleClassifier functionnalities..

In [36]:
...

Ellipsis